<a href="https://colab.research.google.com/github/AlmaBasic/Artificial-intelligence/blob/main/Vjezba_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
import torch

print(torch.__version__)

2.7.0+cu126


In [1]:
import os
import wave
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
import soundfile as sf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls "/content/drive/MyDrive/train/audio"

_background_noise_  dog    four   left	  off	 seven	 three	wow
bed		    down   go	  marvin  on	 sheila  tree	yes
bird		    eight  happy  nine	  one	 six	 two	zero
cat		    five   house  no	  right  stop	 up


In [4]:
path = "/content/drive/MyDrive/train/audio"
classes = os.listdir(path)

In [5]:
classes.remove('_background_noise_')

In [6]:
data = []
classes_ = []
filename = []
duration_length = []


In [7]:
for class_ in classes:
    class_files = os.listdir(os.path.join(path, class_))
    for file_name in class_files:
        full_filename = os.path.join(path, class_, file_name)
        filename.append(file_name)

        y, sr = librosa.load(full_filename, sr=16000)
        duration = librosa.get_duration(y=y, sr=sr)
        classes_.append(class_)

        duration_length.append(duration)

In [8]:
duration_length[:10]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [9]:
filename[:10]

['1365dd89_nohash_0.wav',
 '97e0c576_nohash_2.wav',
 'ac652c60_nohash_0.wav',
 '23ad12d8_nohash_0.wav',
 '161fcca8_nohash_1.wav',
 'a19452d5_nohash_1.wav',
 'b72e58c9_nohash_0.wav',
 'e3411931_nohash_0.wav',
 '1528225c_nohash_3.wav',
 '1706c35f_nohash_3.wav']

In [10]:
df = pd.DataFrame({'Filename': filename, 'Class': classes_, 'Duration': duration_length})

In [11]:
df.head()

,Filename,Class,Duration
0,1365dd89_nohash_0.wav,bed,1.0
1,97e0c576_nohash_2.wav,bed,1.0
2,ac652c60_nohash_0.wav,bed,1.0
3,23ad12d8_nohash_0.wav,bed,1.0
4,161fcca8_nohash_1.wav,bed,1.0


In [12]:
df.shape

(63258, 3)

In [13]:
pip install librosa matplotlib

In [14]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [23]:
audio_path = df.Filename[0]

In [26]:
audio_path

'1365dd89_nohash_0.wav'

In [28]:
if not os.path.exists(audio_path):
    raise FileNotFoundError(f"Fajl nije pronađen: {audio_path}")

FileNotFoundError: Fajl nije pronađen: 1365dd89_nohash_0.wav

In [27]:
y, sr = librosa.load(audio_path)

<ipython-input-27-ce2c310a1eaa>:1: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '1365dd89_nohash_0.wav'

In [22]:
import os
print(os.listdir())

['.config', 'drive', 'sample_data']


In [ ]:
D = librosa.stft(y)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

plt.figure(figsize=(12, 6))
librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar(format='%+2.0f dB')
plt.title('Spektrogram')
plt.tight_layout()
plt.show()

In [ ]:
# 2. Generisanje mel-spektrograma
S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)

# 3. Konverzija u log-skalirani spektrogram (decibeli)
S_dB = librosa.power_to_db(S, ref=np.max)

# 4. Prikaz spektrograma
plt.figure(figsize=(10, 4))
librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-spektrogram')
plt.tight_layout()
plt.show()